In [4]:
from pyspark import SparkContext
sc = SparkContext()

In [5]:
import math

In [6]:
# carregar base de dados
import os.path
fileName = os.path.join('C:\spark\Data', 'DelayedFlights.csv')
numPartitions = 2
rawData = sc.textFile(fileName, numPartitions)


In [11]:
Cancelled = rawData.map(lambda x: x.split(",")[22])
featMonth = rawData.map(lambda x: x.split(",")[2])
featDayofMonth = rawData.map(lambda x: x.split(",")[3])
featDayofWeek = rawData.map(lambda x: x.split(",")[4])

In [ ]:
def Entropia (classe):
    """Calcula a Entropia de Shannon de uma distribuição de dados.

    Args:
        classe (RDD): RDD contendo conjunto de dados a ser calculado a entropia.
                      Valores devem ser categóricos.

    Returns:
        float: valor de Entropia de Shannon calculado para o RDD.
    """
    #counts calcula paralelamente o conteúdo do RDD
    #como tuplas contendo (tipo, quantidade)
    counts = (classe.map(lambda x: (x, 1))
                    .reduceByKey(lambda a,b: a + b))
    # n recebe o valor total de itens do RDD
    n = classe.count()
    
    # probs calcula a probabilidade de cada um dos estados do RDD
    probs = counts.map(lambda x: x[1]/float(n))
    
    # Entropia calcula a entropia do RDD
    ## a função map faz o calculo da Entropia de cada um dos estados
    ## a função reduce faz o somatório da entropia de Shannon
    entropia = (probs.map(lambda p: -p*math.log(p,2))
                     .reduce(lambda a,b: a + b))
    
    # retorna valor escalar referênte a entropia do RDD.
    return entropia

 
def infoGain (feature, classe, H):
    """Calcula o ganho de informação de um atributo em relação a uma classe.

    Args:
        feature (RDD): RDD contendo os conjuntos de dados do atributo a ser
                       calculado o Ganho de Informação
        
        classe (RDD): RDD contendo conjunto de dados da classe
        
        H (float): Entropia da Classe, previamente calculada.

    Returns:
        float: valor de ganho de informação (redução da Entropia) que o atributo fornece sobre a classe
    """
    # calcula paralelamente o conteúdo do RDD
    # como tuplas contendo (tipo, quantidade)
    feat_count = feature.map(lambda x: (x, 1))\
                        .reduceByKey(lambda a,b: a + b)\
                        .collect()
    # calcula as Entropias de um conjunto da classe dado cada um dos estados do atributo   
    entropiasN = [Entropia(classe.zip(feature).filter(lambda x: x[1]==v).map(lambda x: x[0]))  for v,_ in feat_count]
    
    # calcula a quantidade de itens no atributo
    n = classe.count()
        
    # calcula o ganho de informação do atributo.
    ig = H - sum([(f[1]/float(n))*p for f,p in zip(feat_count, entropiasN)])
    
    return ig
    

In [9]:
H =  Entropia (Cancelled)
print H

0.00425591582633


In [15]:
igMonth = infoGain(featMonth, Cancelled, H)
print igMonth

0.00425591582633


In [16]:
igDayofMonth = infoGain(featDayofMonth, Cancelled, H)
print igDayofMonth

0.00425591582633


In [17]:
igDayofWeek = infoGain(featDayofWeek, Cancelled, H)
print igDayofWeek

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 248.0 failed 1 times, most recent failure: Lost task 0.0 in stage 248.0 (TID 1984, localhost, executor driver): java.io.FileNotFoundException: C:\Users\sergi\AppData\Local\Temp\blockmgr-ddae15e5-2a0b-4751-8e15-674bc29273be\22\temp_shuffle_22de9f4c-c0d3-47af-9cea-1fbb9e6fb68a (The system cannot find the path specified)
	at java.io.FileOutputStream.open0(Native Method)
	at java.io.FileOutputStream.open(Unknown Source)
	at java.io.FileOutputStream.<init>(Unknown Source)
	at org.apache.spark.storage.DiskBlockObjectWriter.initialize(DiskBlockObjectWriter.scala:103)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:116)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:237)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWr